In [1]:
# Import dependencies
import nltk
import json
import io
import gzip
import torch
import string
import random
# import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm_notebook
from functools import partial
# import mwparserfromhell

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
PATH_TO_DATA = "/scratch/mz2476/wiki/data/aligned_datasets/"

## Load data from pickle file.
`outlinks` -- names of articles within Wikipedia or smth else that we are not interested in (can be image, graph, etc.). They are **case sensitive**.  
`tokens` -- all lower-cased tokens, cleaned (punctuattions, special symbols removed), no stopwords.  
`mid_level_categories` -- our labels. There are 45 unique categories. Use `MultiLabelBinarizer` from `sklearn` to binarize them (see the code below).

In [3]:
wiki_df = pkl.load(open(PATH_TO_DATA + "en_outlinks_tokens_df.pkl", "rb"))
wiki_df.head()

,QID,title,raw_outlinks,outlinks,raw_tokens,tokens,mid_level_categories
0,Q6199,Anarchism,"[[[Anti-authoritarianism|anti-authoritarian]],...","[Anti-authoritarianism, political philosophy, ...","[anarchism, is, an, anti, authoritarianism, an...","[anarchism, anti, authoritarianism, anti, auth...","[History_And_Society.History and society, Hist..."
1,Q38404,Autism,"[[[Psychiatry]], [[Interpersonal relationship|...","[Psychiatry, Interpersonal relationship, commu...","[autism, is, developmental, disorder, characte...","[autism, developmental, disorder, characterize...","[STEM.Medicine, STEM.Biology, History_And_Soci..."
2,Q101038,Albedo,[[[File:Albedo-e hg.svg|thumb|upright=1.3|The ...,"[File:Albedo-e hg.svg, diffuse reflection, sun...","[sunlight, relative, to, various, surface, con...","[sunlight, relative, various, surface, conditi...","[STEM.Physics, STEM.Space, History_And_Society..."
3,Q173,Alabama,"[[[Coat of arms of Alabama|Coat of arms]], [[N...","[Coat of arms of Alabama, Northern flicker, Di...","[alabama, alabama, is, nicknamed, the, norther...","[alabama, alabama, nicknamed, northern, flicke...",[Geography.Americas]
4,Q41746,Achilles,[[[File:Achilles fighting against Memnon Leide...,[File:Achilles fighting against Memnon Leiden ...,"[dating, to, , three, zero, zero, bc, of, ach...","[dating, three, zero, zero, achilles, killing,...","[Culture.People, Geography.Europe, History_And..."


In [9]:
mlb = MultiLabelBinarizer()
mlb.fit_transform(wiki_df.mid_level_categories)
print(f"Number of categories: {len(mlb.classes_)}, \n\n{mlb.classes_}")

Number of categories: 45, 

['Culture.Architecture' 'Culture.Arts' 'Culture.Broadcasting'
 'Culture.Crafts and hobbies' 'Culture.Entertainment'
 'Culture.Food and drink' 'Culture.Games and toys'
 'Culture.Internet culture' 'Culture.Language and literature'
 'Culture.Media' 'Culture.Music' 'Culture.People'
 'Culture.Performing arts' 'Culture.Philosophy and religion'
 'Culture.Sports' 'Culture.Visual arts' 'Geography.Africa'
 'Geography.Americas' 'Geography.Antarctica' 'Geography.Asia'
 'Geography.Bodies of water' 'Geography.Europe' 'Geography.Landforms'
 'Geography.Maps' 'Geography.Oceania' 'Geography.Parks'
 'History_And_Society.Business and economics'
 'History_And_Society.Education' 'History_And_Society.History and society'
 'History_And_Society.Military and warfare'
 'History_And_Society.Politics and government'
 'History_And_Society.Transportation' 'STEM.Biology' 'STEM.Chemistry'
 'STEM.Engineering' 'STEM.Geosciences' 'STEM.Information science'
 'STEM.Mathematics' 'STEM.Medicine' '